In [ ]:
from __future__ import division                                                 
from __future__ import print_function                                           
from __future__ import absolute_import 

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D              
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.utils import np_utils
from keras.datasets import mnist
from keras.optimizers import SGD, Adam
from keras.callbacks import Callback as Callback
from keras import regularizers

In [ ]:
np.random.seed(1671)
K.set_image_dim_ordering('tf')  

In [ ]:
NUM_CLASSES = 10
NUM_EPOCH = 20
BATCH_SIZE = 128   
OPTIMIZER = Adam()
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
DROPOUT_RATIO = 0.3

INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, 1)  

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
X_train /= 255
X_test /= 255

X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

In [ ]:
Y_train = np_utils.to_categorical(y_train, NUM_CLASSES)
Y_test = np_utils.to_categorical(y_test, NUM_CLASSES)

In [ ]:
model = Sequential()
model.add(Conv2D(20, kernel_size=5, padding='same', input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
print(model.output_shape)

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
print(model.output_shape)

model.add(Dropout(DROPOUT_RATIO))
print(model.output_shape)

model.add(Conv2D(50, kernel_size=5, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(DROPOUT_RATIO))
print(model.output_shape)

model.add(Flatten())
model.add(Dense(500, kernel_regularizer=regularizers.l2(0.001)))
model.add(Activation('relu'))

print(model.output_shape)
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
                      optimizer=OPTIMIZER,
                      metrics=['accuracy'])

In [ ]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.last_epoch_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.last_epoch_start)
        
    def on_train_end(self, logs={}):
        print('\n==> Total time elapsed: ', self.total_time())
        
    def total_time(self): 
        return sum(self.times)

In [ ]:
timeHistory = TimeHistory()

In [ ]:
history = model.fit(X_train, Y_train,
                   batch_size=BATCH_SIZE, 
                   epochs=NUM_EPOCH,
                   callbacks=[timeHistory],
                   verbose=1,
                   validation_split=0.2)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=1)
print("Test score:", score[0])
print("Test accuracy:", score[1])

In [ ]:
def show_bad_only(x, y, preds):
    fig = plot.figure(figsize=(10,20))
    for i in range(10):
        c = 1
        for (image, label, pred) in zip(x, y, preds):
            prediction, actual = np.argmax(pred), np.argmax(label)

            if prediction != i:
                continue

            if i != actual:
                subplot = fig.add_subplot(10,6, i*6+c)
                subplot.set_xticks([])
                subplot.set_yticks([])
                subplot.set_title('pred %d / act %d' % (prediction, actual))
                subplot.imshow(image.reshape((28,28)), vmin=0, vmax=1,cmap=plot.cm.gray_r)

                c += 1
                if c > 6:
                    break

In [ ]:
import matplotlib.pyplot as plot
%matplotlib inline

predictions = model.predict(x=X_test)
show_bad_only(X_test, Y_test, predictions)